In [1]:
import time
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the list of cryptocurrencies
cryptocurrencies = [
    "BTC", "ETH", "XRP", "BCH", "LTC", "EOS", "BNB", "USDT", "LINK", "DOT",
    "ADA", "TRX", "XLM", "NEO", "ATOM", "XMR", "DASH", "XTZ", "ZEC", "UNI"
]

# Define the list of exchanges
exchanges = ["wazirx", "coinswitch", "mudrex", "coindcx"]

# Function to fetch price data from the specified exchange API
def fetch_price(exchange, api_endpoint):
    url = "https://api.wazirx.com/api/v2/market-status"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from {exchange}: {e}")
        return None
    except ValueError as e:
        print(f"Error decoding JSON from {exchange}: {e}")
        print("Response content:", response.content)
        return None
# Example usage:
data = fetch_price("wazirx", "https://api.wazirx.com/api/v2/market-status")
# data = fetch_price("coindcx", "https://api.coindcx.com//exchange/ticker")
# Function to extract price for each cryptocurrency from the response

def extract_prices(data, cryptocurrency):
    prices = {}
    for exchange, info in data.items():
        if cryptocurrency in info:
            prices[exchange] = info[cryptocurrency]
    return prices

# Function to fetch all prices
def fetch_all_prices():
    all_prices = {}
    for exchange in exchanges:
        data = fetch_price(exchange,'https://api.wazirx.com/api/v2/market-status')
        if data:
            for cryptocurrency in cryptocurrencies:
                prices = extract_prices(data, cryptocurrency)
                if cryptocurrency not in all_prices:
                    all_prices[cryptocurrency] = {}
                all_prices[cryptocurrency][exchange] = prices.get(exchange, "N/A")
    return all_prices

# Function to update Google Sheet with prices
def update_google_sheet(sheet_name, all_prices):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Project/Web scraping Crypto data/gold-virtue-411410-f7c56448642a.json', scope)
    client = gspread.authorize(credentials)
    sheet = client.open('/content/drive/MyDrive/Project/Web scraping Crypto data/Crypto Prices.gsheet').sheet1
    header = ["Cryptocurrency"] + exchanges
    sheet.clear()
    sheet.append_row(header)
    for cryptocurrency, prices in all_prices.items():
        row = [cryptocurrency]
        for exchange, price in prices.items():
            row.append(price)
        sheet.append_row(row)

# Main function
def main():
    while True:
        all_prices = fetch_all_prices()
        update_google_sheet("/content/drive/MyDrive/Project/Web scraping Crypto data/Crypto Prices.gsheet", all_prices)
        print("Updated Google Sheet with latest prices.")
        time.sleep(60)  # Wait for 60 seconds before updating again

if __name__ == "__main__":
    main()

SpreadsheetNotFound: <Response [200]>